<a href="https://colab.research.google.com/github/Group4-s2025/DATA-6550-Algorithms/blob/main/Module_4_Project_Group_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Housing Discrimination and Algorithmic Bias
## Module 4: Algorithmic Bias
### By: Savi Alwis, Moses Kipkemboi, Muqtasid Qureshi, Pallavi Suram

In [ ]:
# Load libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# Load dataset
url = "https://raw.githubusercontent.com/savi3alwis/DATA-6550-Algorithms/refs/heads/main/Data/mortgage_covenant_data.csv"
df = pd.read_csv(url)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100931 entries, 0 to 100930
Data columns (total 14 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   activity_year                      100931 non-null  int64  
 1   census_tract                       99915 non-null   float64
 2   derived_race                       100931 non-null  object 
 3   action_taken                       100931 non-null  int64  
 4   loan_amount                        100931 non-null  float64
 5   property_value                     83491 non-null   float64
 6   income                             88200 non-null   float64
 7   interest_rate                      78125 non-null   float64
 8   tract_minority_population_percent  100931 non-null  float64
 9   tract_to_msa_income_percentage     100931 non-null  float64
 10  denial_reason-1                    100931 non-null  int64  
 11  covenant_count                     1009